In [32]:
import sqlite3
import yaml
import numpy as np

class DBOperate:
    def __init__(self, db_path):
        self.conn = sqlite3.connect(db_path)
        self.cursor = self.conn.cursor()

    def create_table(self, table_name, fields):
        field_defs = [f"{name} {type}" for name, type in fields.items()]
        field_defs_str = ", ".join(field_defs)
        sql = f"CREATE TABLE IF NOT EXISTS {table_name} ({field_defs_str});"
        self.cursor.execute(sql)

    def insert_data(self, table_name, fields, data):
        placeholders = ", ".join(["?"] * len(fields))
        sql = f"INSERT INTO {table_name} ({', '.join(fields)}) VALUES ({placeholders})"
        print(f"Inserting into {table_name}: {sql}")
        print(f"Fields: {fields}")
        print(f"Number of fields: {len(fields)}")
        print(f"First row of data: {data[0]}")
        print(f"Number of columns in data: {len(data[0])}")
        self.cursor.executemany(sql, data)
        self.conn.commit()

    def close(self):
        self.conn.close()

# 读取配置文件
with open('config.yaml', 'r', encoding='utf-8') as f:
    DB_CONFIG = yaml.safe_load(f)

# 提取配置信息
db_path = DB_CONFIG["db_path"]
TABLE = DB_CONFIG["TABLE"]
TABLE_Values = DB_CONFIG["TABLE_Values"]

dboperate = DBOperate(db_path)

print("建表中...")
# 创建表
for table_name, table_info in TABLE.items():
    fields = {k: v[1] for k, v in table_info['field'].items()}
    dboperate.create_table(table_name, fields)

print("插入数据中...")
# 插入数据
for table_name, table_data in TABLE_Values.items():
    fields = list(table_data["field"].keys())
    table_val = np.array(list(table_data["field"].values()), dtype=object).T.tolist()
    dboperate.insert_data(table_name, fields, table_val)

dboperate.close()
print("数据插入完成")


建表中...
插入数据中...
Inserting into people_info: INSERT INTO people_info (name, gender, ethnicity, education_level, id_number, phone_number, registered_address, residential_address, category, judgment_level, village, community) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
Fields: ['name', 'gender', 'ethnicity', 'education_level', 'id_number', 'phone_number', 'registered_address', 'residential_address', 'category', 'judgment_level', 'village', 'community']
Number of fields: 12
First row of data: ['张三', 1, '汉族', 6, 123456789012345678, 13800138000, '北京市海淀区', '北京市朝阳区', '普通人员', 1, '海淀村', '海淀社区']
Number of columns in data: 12
Inserting into camera_info: INSERT INTO camera_info (camera_model, coordinates, osd) VALUES (?, ?, ?)
Fields: ['camera_model', 'coordinates', 'osd']
Number of fields: 3
First row of data: ['型号A', '39.9042, 116.4074', 'OSD信息1']
Number of columns in data: 3
Inserting into person_recognition: INSERT INTO person_recognition (time, name, camera_id, big_image, small_image, vl_outpu